# Python Decorators

### Why decorators?

Decorators are a flexible means to **extend the functionality of a callable** (function,
method, or class). Useful e.g. for logging, access control, memoization, and performance
measurement

Allows to adhere to **best practices** such as modularity and separation of concerns by allowing
functionality to be added to existing code in a clean, efficient, and reusable manner.

This is achieved **without the need for inheritance or modifying the original callables**'s
code, thus keeping the code [DRY](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)
(Don't Repeat Yourself) and adhering to the [open/closed
principle](https://www.pythontutorial.net/python-oop/python-open-closed-principle/).

$f(\cdot) \mapsto F(f(\cdot))$: Think of decorators of as **functions that take another
function** as an argument and return a new function, thereby wrapping the original
function in order to provide additional functionality.

### Example: Measuring Function Execution Time

Say we want to measure the execution time of any function which we choose to decorate.

The example below illustrates the elegance and power of decorators in adding
functionality (in this case, performance measurement) in a modular and non-intrusive
manner. Decorators are a testament to Python's commitment to clean, readable, and
expressive code.


In [ ]:
import time

def timeit_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        running_time = end_time - start_time
        return (running_time, result)

    return wrapper

In [ ]:
@timeit_decorator
def sample_function(n):
    """Function to sum the first n natural numbers."""
    return sum(range(1, n + 1))

running_time, result = sample_function(1000000)
print(f"Running time: {running_time*1000:.3f} ms, Result: {result}")


### Example: Standard library's `dataclass`

Decorator applied to class that generates common special methods (`__init__`, `__repr__`, `__eq__`, etc)

Helps to write cleaner, more concise code by eliminating the need to write code that
e.g. initializes or compares instances of a class

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Person:
    name: str
    age: int

# Creating an instance of Person
person1 = Person("John Doe", 30)

# The __repr__ method is automatically generated, so this will print a nicely formatted
# representation of the object
print(person1)

# Comparison is automatically handled
person2 = Person("Jane Doe", 29)
person3 = Person("John Doe", 30)
print("person1 == person2:", person1 == person2)
print("person1 == person3:", person1 == person3)

# Accessing attributes
print(f"{person1.name} is {person1.age} yo")

### Example: Standard library's `partial`

Decorator applied to functions used to create *partial* functions. 

Partial functions allow pre-setting some of the arguments of the original function and generate a new function with those arguments already filled in.


In [ ]:
from functools import partial


def multiply(x, y, z):
    return f"{x} * {y} * {z} = {x * y * z}"


# Another form of using a decorator (no '@')
two_times = partial(multiply, 2)
print("times_two(3, 4):  ", two_times(3, 4))

# Bind arbitrary arguments using keyword arguments
two_times_six = partial(multiply, 2, z=6)
print("two_times_five(3):", two_times_six(3))

### Example: Third-party library `tenacity`

Consider a scenario where we're making requests to a **remote, failure-prone service**

`tenacity` enables the application to withstand and recover from temporary issues $\Rightarrow$ more resilient applications and **much improved user experience**

In [ ]:
class UnstableRequestsSimulator:
    def __init__(self, succeed_at_attempt):
        self._succeed_at_attempt = succeed_at_attempt
        self._attempt = 0

    def __call__(self):
        self._attempt += 1
        if self._succeed_at_attempt is None or self._attempt < self._succeed_at_attempt:
            raise Exception("Request failed due to unstable network connection")

        self._attempt = 0
        return "Success!"

make_request_3 = UnstableRequestsSimulator(succeed_at_attempt=3)

for i in range(10):
    try:
        make_request_3()
        print(f"Attempt {i + 1}: OK")
    except:
        print(f"Attempt {i + 1}: Failed")


In [ ]:
from tenacity import retry, stop_after_attempt, wait_fixed

make_request_3 = UnstableRequestsSimulator(succeed_at_attempt=3)

@retry(stop=stop_after_attempt(5), wait=wait_fixed(1))
def make_request_with_retry():
    print("Making request...")
    return make_request_3()

result = make_request_with_retry()
print(result)

`tenacity` is highly configurable, worth having a look at the [docs](https://tenacity.readthedocs.io/en/latest/)